<a href="https://colab.research.google.com/github/Fhashemi91/NLP-assignment-2/blob/main/NLP_AS1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installing Dependencies

In [ ]:
!pip install nltk
!pip install pandas
!pip install sklearn
!pip install xgboost

### Importing dependencies

In [ ]:
import nltk
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import re
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import f1_score, accuracy_score
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger') 


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kimge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kimge\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\kimge\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kimge\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\kimge\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

### Loading News Dataset

We are intrested in the title and date columns.
We are also interested in records with AAPL ticker.

In [ ]:
# loading dataframe from csv file
news = pd.read_csv('./us_equities_news_dataset.csv')
# Dropping irrelevant columns
news = news.drop(['content', 'url', 'article_id', 'provider'], axis = 1) 

# Filtering for AAPL news
news = news[news.ticker=='AAPL']
news

,id,ticker,title,category,release_date
49183,270698,AAPL,JPMorgan cautious ahead of Apple earnings,news,2020-01-28
49184,270699,AAPL,FAANG s Fall but Get Some Wall Street Love,news,2020-01-28
49185,270700,AAPL,Wall Street tumbles as virus fuels economic worry,news,2020-01-28
49186,270701,AAPL,Earnings Watch Apple and AMD to take earnings...,news,2020-01-28
49187,270702,AAPL,Day Ahead Top 3 Things to Watch for Jan 28,news,2020-01-28
...,...,...,...,...,...
69409,290924,AAPL,Waiting For Direction On The Markets,opinion,2012-07-16
69410,290925,AAPL,Mid Year Update U S And Canadian Stock Marke...,opinion,2012-07-19
69411,290926,AAPL,Summer Heat Scorches Europe And U S,opinion,2012-07-23
69412,290927,AAPL,Apple Earnings Preview Quarterly Dip On Deck,opinion,2012-07-23


In [ ]:
pd.read_csv('./AAPL.csv')

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-01-15,2.877500,3.004286,2.858929,2.977857,2.542589,1831634000
1,2009-01-16,3.010714,3.013571,2.871429,2.940357,2.510570,1047625600
2,2009-01-20,2.926071,2.928571,2.792857,2.792857,2.384630,919914800
3,2009-01-21,2.835357,2.960000,2.832500,2.958214,2.525818,1089270000
4,2009-01-22,3.144286,3.214286,3.065000,3.155714,2.694449,1409528400
...,...,...,...,...,...,...,...
3435,2022-09-08,154.639999,156.360001,152.679993,154.460007,154.460007,84923800
3436,2022-09-09,155.470001,157.820007,154.750000,157.369995,157.369995,68028800
3437,2022-09-12,159.589996,164.259995,159.300003,163.429993,163.429993,104956000
3438,2022-09-13,159.899994,160.539993,153.369995,153.839996,153.839996,122656600


In [ ]:
# Loading dataframe from csv file
stocks = pd.read_csv('./AAPL.csv').drop(['High', 'Low', 'Adj Close', 'Volume'], axis=1)
stocks

,Date,Open,Close
0,2009-01-15,2.877500,2.977857
1,2009-01-16,3.010714,2.940357
2,2009-01-20,2.926071,2.792857
3,2009-01-21,2.835357,2.958214
4,2009-01-22,3.144286,3.155714
...,...,...,...
3435,2022-09-08,154.639999,154.460007
3436,2022-09-09,155.470001,157.369995
3437,2022-09-12,159.589996,163.429993
3438,2022-09-13,159.899994,153.839996


We are labeling the records with close>=open as 1 and others as 0

In [ ]:
# Creating a binary feature for movement (close-open) in the stock price
stocks.loc[stocks["Close"] >= stocks["Open"], "movement"] = 1
stocks.loc[stocks["Close"] < stocks["Open"], "movement"] = 0
stocks

,Date,Open,Close,movement
0,2009-01-15,2.877500,2.977857,1.0
1,2009-01-16,3.010714,2.940357,0.0
2,2009-01-20,2.926071,2.792857,0.0
3,2009-01-21,2.835357,2.958214,1.0
4,2009-01-22,3.144286,3.155714,1.0
...,...,...,...,...
3435,2022-09-08,154.639999,154.460007,0.0
3436,2022-09-09,155.470001,157.369995,1.0
3437,2022-09-12,159.589996,163.429993,1.0
3438,2022-09-13,159.899994,153.839996,0.0


### Setup preprocessing ###

<!-- Next we get to preprocess our data by:
* Lowercasing
* Removing stop words
* Removing @words
* Changing U S and S P to US and SP
* Removing punctuations, numbers and special characters
* Lemmatization by part of speech
* removing trailing white spaces -->

In [ ]:
# Lemmatizer object
lemma = WordNetLemmatizer()


def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Remove "@name"
    - Isolate and remove punctuations 
    - Remove other special characters
    - Remove stop words 
    - Remove trailing whitespace
    """
    # lowercasing
    s = s.lower()
    # changing u s and s p to us and sp
    s = re.sub(r'(^|\s)(u\ss)($|\s)', ' us ', s)
    s = re.sub(r'(^|\s)(s\sp)($|\s)', ' sp ', s)
    # removing stopwords
    s = " ".join([word for word in s.split() if word not in stopwords.words('english')])
    # remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # remvoing punctuations
    s = re.sub(r'([\'\"\.\(\)\!\?\\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?\!]', ' ', s)
    # removing numbers
    s = re.sub(r'\d+', ' ', s)
    # removing some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # lemmatization    
    s = " ".join([lemma.lemmatize(w,pos='n') if t.startswith('N') else # noun
                  lemma.lemmatize(w,pos='v') if t.startswith('V') else # verb
                  lemma.lemmatize(w,pos='a') if t.startswith('JJ') else # adjective
                  lemma.lemmatize(w,pos='r') if t.startswith('R') else # adverbs
                  
                  w for w,t in pos_tag(word_tokenize(s))])
    # removing trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()
    return s
# test
text_preprocessing('JPMorgan cautious ahead of Apple earnings u s @here 2.')

'jpmorgan cautious ahead apple earnings us'

### Process titles

In [ ]:
# Applying preprocessing
news['processed_titles'] = news['title'].apply(text_preprocessing)
news

,id,ticker,title,category,release_date,processed_titles
49183,270698,AAPL,JPMorgan cautious ahead of Apple earnings,news,2020-01-28,jpmorgan cautious ahead apple earnings
49184,270699,AAPL,FAANG s Fall but Get Some Wall Street Love,news,2020-01-28,faang fall get wall street love
49185,270700,AAPL,Wall Street tumbles as virus fuels economic worry,news,2020-01-28,wall street tumble virus fuel economic worry
49186,270701,AAPL,Earnings Watch Apple and AMD to take earnings...,news,2020-01-28,earnings watch apple amd take earnings stage a...
49187,270702,AAPL,Day Ahead Top 3 Things to Watch for Jan 28,news,2020-01-28,day ahead top thing watch jan
...,...,...,...,...,...,...
69409,290924,AAPL,Waiting For Direction On The Markets,opinion,2012-07-16,wait direction market
69410,290925,AAPL,Mid Year Update U S And Canadian Stock Marke...,opinion,2012-07-19,mid year update us canadian stock market sector
69411,290926,AAPL,Summer Heat Scorches Europe And U S,opinion,2012-07-23,summer heat scorch europe us
69412,290927,AAPL,Apple Earnings Preview Quarterly Dip On Deck,opinion,2012-07-23,apple earnings preview quarterly dip deck


In [ ]:
# Merging articles of the same day to one long string
c_news = news.groupby('release_date')['processed_titles'].apply(list).reset_index(name='processed_titles')
c_news['processed_titles']= c_news['processed_titles'].apply(' '.join)
c_news

,release_date,processed_titles
0,2012-07-16,wait direction market
1,2012-07-19,mid year update us canadian stock market sector
2,2012-07-23,summer heat scorch europe us apple earnings pr...
3,2012-07-24,market bait switch
4,2012-07-27,aapl fall tree
...,...,...
2262,2020-01-24,stock buy list already run preview tech sector...
2263,2020-01-25,apple tv already million customer
2264,2020-01-26,western digital wdc report q earnings expect a...
2265,2020-01-27,apple set beat q earnings estimate tech etfs b...


In [ ]:
# Merging news with stocks
dataset = pd.merge(c_news, stocks, left_on='release_date', right_on='Date')
dataset

,release_date,processed_titles,Date,Open,Close,movement
0,2012-07-16,wait direction market,2012-07-16,21.611429,21.675358,1.0
1,2012-07-19,mid year update us canadian stock market sector,2012-07-19,21.831429,21.940001,1.0
2,2012-07-23,summer heat scorch europe us apple earnings pr...,2012-07-23,21.228571,21.565357,1.0
3,2012-07-24,market bait switch,2012-07-24,21.692142,21.461430,0.0
4,2012-07-27,aapl fall tree,2012-07-27,20.536072,20.898571,1.0
...,...,...,...,...,...,...
1735,2020-01-22,zacks market edge highlight vegn apple microso...,2020-01-22,79.644997,79.425003,0.0
1736,2020-01-23,gpu adoption epyc deal win aid amd q earnings ...,2020-01-23,79.480003,79.807503,1.0
1737,2020-01-24,stock buy list already run preview tech sector...,2020-01-24,80.062500,79.577499,0.0
1738,2020-01-27,apple set beat q earnings estimate tech etfs b...,2020-01-27,77.514999,77.237503,0.0


In [ ]:
# train-test split
X_train,X_test, y_train,y_test = train_test_split(dataset['processed_titles'],
                                                  dataset['movement'],
                                                  test_size=0.33,
                                                  random_state=42)

We vectorize our titles to be able to use them with quantitative models. For this we are using a simple CountVectorizer, also looking for bigrams and limit the result to 100 most informative features.

In [ ]:
# Vectorizer for single words and bigrams, keeping the top 100 features, creating a bag of words considering word occurences
vectorizer = CountVectorizer(ngram_range=(1,2), max_features=100) 
vectorizer.fit(X_train)

CountVectorizer(max_features=100, ngram_range=(1, 2))

In [ ]:
# Turning count vectors to dataframes 
colNames = vectorizer.get_feature_names_out() 
X_train_c = pd.DataFrame(vectorizer.transform(X_train).toarray(),columns=colNames)
X_test_c = pd.DataFrame(vectorizer.transform(X_test).toarray(),columns=colNames)

In [ ]:
# Investigating label balance
dataset.movement.describe()

count    1740.000000
mean        0.526437
std         0.499444
min         0.000000
25%         0.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: movement, dtype: float64

In [ ]:
# Training an xgboost classifier
model = XGBClassifier(objective='binary:logistic')
model.fit(X_train_c, y_train)
predictions = model.predict(X_test_c)

C:\Users\kimge\anaconda3\lib\site-packages\xgboost\data.py:173: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


We use f1 score for evaluation of our binary predictions

In [ ]:
f1_score(y_test, predictions)

0.5328836424957842

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF_model = RandomForestClassifier(n_estimators=275, criterion='gini', max_features='auto')
RF_model = RF_model.fit(X_train_c, y_train)
RF_predictions = RF_model.predict(X_test_c)

C:\Users\kimge\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [ ]:
f1_score(y_test, RF_predictions)

0.5391014975041597